# Data preporation

## Simple Tokenizer

In [1]:
import urllib.request
url = ("https://raw.githubusercontent.com/rasbt/"
 "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
 "the-verdict.txt")
file_path = "the-verdict.txt"
urllib.request.urlretrieve(url, file_path)

('the-verdict.txt', <http.client.HTTPMessage at 0x1c7c64abb10>)

In [2]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
 raw_text = f.read()
print("Total number of character:", len(raw_text))
print(raw_text[:99])

Total number of character: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [3]:
import re

In [4]:
items = [item for item in re.split(r'([,.?_!"()\']|--|\s)', raw_text) if item.strip()]

In [5]:
all_words = sorted(set(items))
vocab = {word:i for i, word in enumerate(all_words)}

In [6]:
class SimpleTokenizerV1():

    def __init__(self, vocab):
        self.str_to_int = vocab.copy()
        self.int_to_str = {idx:item for item, idx in vocab.items()}

    def encode(self, text):
        items = [item for item in re.split(r'([,.?_!"()\']|--|\s)', text) if item.strip()]
        return [self.str_to_int[item] for item in items]

    def decode(self, tokens):
        text = ' '.join(self.int_to_str[token] for token in tokens)
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text) 
        return text

In [7]:
simple_tokenizer = SimpleTokenizerV1(vocab)

In [8]:
tokens = simple_tokenizer.encode(raw_text[:98])

In [9]:
decoded_tokens = simple_tokenizer.decode(tokens)

In [10]:
raw_text[:98]

'I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no'

In [11]:
text = """"It's the last he painted, you know," Mrs. Gisburn said with pardonable pride."""
tokens = simple_tokenizer.encode(text)
print(tokens)
print(simple_tokenizer.decode(tokens))

[1, 58, 2, 872, 1013, 615, 541, 763, 5, 1155, 608, 5, 1, 69, 7, 39, 873, 1136, 773, 812, 7]
" It' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.


In [13]:
vocab['<|unk|>'] = len(vocab)
vocab['<|endoftext|>'] = len(vocab)

In [17]:
class SimpleTokenizerV2():

    def __init__(self, vocab):
        self.str_to_int = vocab.copy()
        self.int_to_str = {idx:item for item, idx in vocab.items()}
        self.unknown_token = '<|unk|>'

    def encode(self, text):
        items = [item for item in re.split(r'([,.?_!"()\']|--|\s)', text) if item.strip()]
        items = [item if item in self.str_to_int else self.unknown_token for item in items]
        return [self.str_to_int[item] for item in items] 

    def decode(self, tokens):
        text = ' '.join(self.int_to_str[token] for token in tokens)
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text) 
        return text

In [18]:
simple_tokenizer_v2 = SimpleTokenizerV2(vocab)

In [20]:
text = """"It's the last he painted, you know," Mrs. Gisburn said with pardonable pride. hello"""
tokens = simple_tokenizer_v2.encode(text)
print(tokens)
print(simple_tokenizer_v2.decode(tokens))

[1, 58, 2, 872, 1013, 615, 541, 763, 5, 1155, 608, 5, 1, 69, 7, 39, 873, 1136, 773, 812, 7, 1159]
" It' s the last he painted, you know," Mrs. Gisburn said with pardonable pride. <|unk|>


In [28]:
import tiktoken

In [29]:
tokenizer = tiktoken.get_encoding('gpt2')

In [30]:
text = (
 "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
 "of someunknownPlace."
)
integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [31]:
tokenizer.decode(integers)

'Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.'

In [35]:
for item in tokenizer.encode( 'Akwirw ier'):
    print(tokenizer.decode([item])) 

Ak
w
ir
w
 
ier


## Dataset creation

In [42]:
import torch
from torch.utils.data import Dataset, DataLoader

In [38]:
enc_text = tokenizer.encode(raw_text)

In [56]:
class GPTDatasetV1(Dataset):

    def __init__(self, text, tokenizer, max_len, stride):
        self.input_ids = []
        self.target_ids = []

        tokens = tokenizer.encode(text)
        for i in range(0, len(tokens)-max_len, stride):
            input_chunk = tokens[i:i+max_len]
            target_chunk = tokens[i+1:i+max_len+1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids) 

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [57]:
def create_dataloader_v1(text, max_len=256, 
                         stride=128, 
                         batch_size=4, 
                         shuffle=True, 
                         drop_last=True, 
                         num_workers=0):
    tokenizer = tiktoken.get_encoding('gpt2')

    dataset = GPTDatasetV1(text, tokenizer, max_len, stride)

    dataloader = DataLoader(
        dataset=dataset, 
        batch_size=batch_size,
        shuffle=shuffle, 
        drop_last=drop_last, 
        num_workers=num_workers
    )

    return dataloader

In [58]:
dataloader = create_dataloader_v1(raw_text, max_len=4, batch_size=1, stride=1, shuffle=False)

In [59]:
data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


## Embedding 

In [63]:
vocab_size = 50257
out_dim = 256
max_len = 4                                                                                                                                                   

In [64]:
token_embedding_layer = torch.nn.Embedding(vocab_size, out_dim)

In [70]:
dataloader = create_dataloader_v1(raw_text, max_len=max_len, batch_size=8, stride=max_len, shuffle=False)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Token IDs:\n", inputs)
print("\nInputs shape:\n", inputs.shape)

Token IDs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Inputs shape:
 torch.Size([8, 4])


In [71]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

torch.Size([8, 4, 256])


In [73]:
pos_embedding_layer = torch.nn.Embedding(max_len, out_dim)
pos_embeddings = pos_embedding_layer(torch.arange(max_len))

In [74]:
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

torch.Size([8, 4, 256])
